# Length substitution rates

(c) 2020 Tom Röschinger. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

In [1]:
using Jedi, Plots, Measures

Jedi.default_plotlyjs!()

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7622954599892522170\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2497490254614693929\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7622954599892522170\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14523713023607678463\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-1392002352555670555\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-14523713023607678463\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Plots.PlotlyJSBackend()

In [2]:
N = 1000
f0 = 20/2N
fl = 
Est(l) = 2*(3/4 * l - 5)
F = Jedi.fermi_fitness(f0=f0, E_Star=Est)

fermi_fitness(10, 1.0, 0.01, 0.0, Est)

In [3]:
function fixed_l_Q(rho, F)
    Q_k = zeros(Float64, 33, 101)
    for (j, l_0) in enumerate(8:40)
        F.l = l_0
        for (i, g) in enumerate(0:0.01:1)
            Q_k[j, i] = exp(-1/2*(l_0 * (g-3/4))^2/(3*l_0/16)) * exp(2N/(1+rho)*fitness(2l_0*g, F))
        end
    end
    return Q_k
end

fixed_l_Q (generic function with 1 method)

In [4]:
plot(collect(0:0.01:1), fixed_l_Q(0, F)[15, :])

<!DOCTYPE html>
 
 
 Plots.jl

### Computing substitution rates

In [5]:
# Definition of the length substitution rates. Also computes
function length_rates(l, F, rho=0, epsilon=2)
    F.l = l
    
    # Compute conditional distribution
    Q_k = zeros(Float64, 101)
    for (i, g) in enumerate(0:0.01:1)
        Q_k[i] = exp(-1/2*(l * (g-3/4))^2/(3*l/16)) * exp(2N/(1+rho)*fitness(2l*g, F))
    end
    
    # Normalize Q
    Q_k /= sum(Q_k) * 0.01
    
    g = 0:0.01:1 |> collect
    
    # Add match
    s_m_plus = fitness.((2l .* g), l+1, F) .- fitness.((2l .* g), l, F)
    # Add mismatch
    s_mm_plus = fitness.((2l .* g) .+ epsilon, l+1, F) .- fitness.(2l .* g, l, F)
    u_plus = ((1/4 .* Jedi.kimura_prob.(s_m_plus, N)) .+ (3/4 .* Jedi.kimura_prob.(s_mm_plus, N))) .* Q_k |> sum
    
    # Remove mismatch
    s_mm_minus = fitness.(2l .* g .- epsilon, l-1, F) .- fitness.(2l .* g, l, F)
    # Remove match
    s_m_min = fitness.(2l .* g, l-1, F) .- fitness.(2l .* g, l, F)
    u_minus = ((1 .- g) .* Jedi.kimura_prob.(s_m_min, N) .+ g .* Jedi.kimura_prob.(s_mm_minus, N)) .* Q_k |> sum
    return u_plus, u_minus
end

length_rates (generic function with 3 methods)

Use this function to compute the substitution rates.

In [6]:
up = zeros(31)
down = zeros(31)
for (i, l) in enumerate(10:40)
    up[i], down[i] = length_rates(l, F)
end

Using detailed balance to compute $P(l)$.

In [12]:
p_l = zeros(31)
p_l[1] = 0.1
for i in 2:31
    p_l[i] = p_l[i-1] * up[i-1] / down[i]
end


In [13]:
plot(10:40, p_l/sum(p_l), xlabel="l", yformatter=_->"", label="ρ=0", title="P(l)", titlefontsize=12, top_margin=4mm, linewidth=2)

<!DOCTYPE html>
 
 
 Plots.jl

Repeat the computation for 

In [14]:
up = zeros(31)
down = zeros(31)
for (i, l) in enumerate(10:40)
    up[i], down[i] = length_rates(l, F, 1)
end

p_l2 = zeros(31)
p_l2[1] = 0.1
for i in 2:31
    p_l2[i] = p_l2[i-1] * up[i-1] / down[i]
end

plot!(10:40, p_l2/sum(p_l2), label="ρ=1")

<!DOCTYPE html>
 
 
 Plots.jl

There does not seem to be a big difference! Let's include a length cost.

In [15]:
F_2 = Jedi.fermi_fitness(f0=f0, E_Star=Est, fl=0.25/2N)

fermi_fitness(10, 1.0, 0.01, 0.000125, Est)

In [16]:
up = zeros(31)
down = zeros(31)
for (i, l) in enumerate(10:40)
    up[i], down[i] = length_rates(l, F_2)
end

Using detailed balance to compute $P(l)$.

In [17]:
p_l = zeros(31)
p_l[1] = 0.1
for i in 2:31
    p_l[i] = p_l[i-1] * up[i-1] / down[i]
end

In [18]:
plot(10:40, p_l/sum(p_l), xlabel="l", yformatter=_->"", label="ρ=0", title="P(l)", titlefontsize=12, top_margin=4mm, linewidth=2)

<!DOCTYPE html>
 
 
 Plots.jl

In [19]:
up = zeros(31)
down = zeros(31)
for (i, l) in enumerate(10:40)
    up[i], down[i] = length_rates(l, F_2, 1)
end

p_l2 = zeros(31)
p_l2[1] = 0.1
for i in 2:31
    p_l2[i] = p_l2[i-1] * up[i-1] / down[i]
end

plot!(10:40, p_l2/sum(p_l2), label="ρ=1")

<!DOCTYPE html>
 
 
 Plots.jl

Here we can see two things. First that the distribution is peaked around lengths that are much higher than the ones which minimize the load. And also we can see how the peak shifts to higher lengths when the system is in non-equilibrium.